# Identifying performance bottlenecks in a multiple queries


In [2]:
import pandas as pd

from plotly.offline import init_notebook_mode
from sqlalchemy import create_engine

from query_flow.parsers.postgres_parser import PostgresParser
from query_flow.vizualizers.query_vizualizer import QueryVizualizer
from plotly.offline import plot, iplot

**Slow queries are common and can be caused by various reasons. This makes optimizing queries extremely hard. The main reason multi-queries optimization is even harder is that one query may affect the other. Thus, it is critical to make sure that we take other queries into consideration.** 

Using QueryFlow we can visualize the queries’ sub-expressions characteristics. Since the execution time of a sub-expression can be caused by several reasons, we might need to visualize the cardinality (actual_rows), duration (actual_duration), and other statistics like whether the operation spilled to disk.

We can use QueryFlow as is and visualize two separate Sankies for each of our queries. But we can make it even easier if we construct one Sankey for multiple queries. To do so we need to modify both the parsing and the visualization algorithm. 

In the parsing algorithm, we add a step that checks if a sub-expression was already seen and assigns the current sub-expression with that expression id. To make this possible instead of calculating the node id as an incremental integer, we use a hash function on operation_type, label, and label_metadata. This allows us to see our queries in a compact form as our sub-expressions “share” nodes. The modification for the algorithm can be seen in figure 50.

To make differentiating the two queries, we assign each query its color. There is no change required in the enrichment algorithm, as this is done in the visualization phase. The algorithm calculates the hue (color) for each metric in step 5 and then modifies each rows’ hue according to its corresponding metric. The modification for the visualization algorithm can be seen in figure 51

To illustrate the gist of the MQO problem, we use two simple queries. The first query is the same example as in 4.2. The query can be seen in figure 52. 

 
	¬¬              Figure 52 – Identify Performance Bottleneck First Query

The second query we use is the same example as in chapter 3. The query can be seen in figure 53. 


In [3]:
con_str = 'postgresql:///etrabelsi_thesis'
query_renderer = QueryVizualizer(parser=PostgresParser(is_compact=True))
with create_engine(con_str).connect() as con:
    con.execute("DROP INDEX if exists crew_index")
    con.execute("DROP INDEX if exists titles_index")
query1 = """
SELECT titles.title_id
FROM titles
INNER JOIN crew ON crew.title_id = titles.title_id
INNER JOIN people ON people.person_id = crew.person_id
WHERE genres like '%Comedy%' 
  AND name in ('Owen Wilson', 'Adam Sandler', 'Jason Segel')
"""


query2 = """
SELECT titles.title_id
FROM titles
WHERE genres = 'Comedy' 
UNION ALL
SELECT titles.title_id
FROM titles
WHERE genres = 'Action' 
"""

queries = [query2, query1]
flow_df = query_renderer.get_flow_df(queries, con_str=con_str)
query_renderer.vizualize(flow_df, metrics=["actual_duration"], title="multiple queries optimization",open_=False)    

We can see that the same logical operations can share the same nodes and each query gets its own color (hue), the first query is  represented in yellow and the second query is  represented in green.

In terms of optimizing our queries, we can see that only a few rows are retrieved from the People* ⋈ Crew due to the filter on the People relation and the same goes for Title relation filtered, which indicates there might be a good candidate for optimization. 

To get more information, we created another Sankey that represents the duration of our example, and it can be seen in the next cell.


In [20]:
query_renderer.vizualize(flow_df, metrics=["actual_rows"], title="multiple queries optimization",open_=False)

The total query duration of both queries is 7.5 seconds (from the “Total time”); we can see that the longest operation (3.6 seconds) is the People* ⋈ Crew.

We can improve our query performance in multiple ways. The easiest one is to add a new index to the Crew relation using the person_id column. The reason an index improves our query execution time is that there are a lot of rows that can be skipped in the Crew scan. This allows us to use PostgreSQL’ Hash Join in a performant way (Index Join).  

The index creation query on the crew relation with the person_id column can be seen in the next cell. 


In [38]:
with create_engine(con_str).connect() as con:
    con.execute("CREATE INDEX if not exists crew_index ON crew(person_id)")
flow_df = query_renderer.get_flow_df(queries, con_str=con_str)
query_renderer.vizualize(flow_df, metrics=["actual_duration"], title="multiple queries optimization",open_=False)

The total query duration is 4.5 seconds (from the “Total time” of the last node); we can see  that it improved both the People* ⋈ Crew and the scan on the Crew relation.  

Performance optimization is an iterative process. To illustrate the MQO in the best way, we continue to improve our queries. We see that both queries uses Title scan and have predicates. We add an index to the Titles relation using the title_id and genres columns.The corresponding Sankey that represents the duration of our query can be seen in the next cell.

In [24]:
with create_engine(con_str).connect() as con:
    con.execute("CREATE INDEX if not exists titles_index ON titles (title_id,genres) WHERE genres like '%%Comedy%%'")
flow_df = query_renderer.get_flow_df(queries[::-1], con_str=con_str)
query_renderer.vizualize(flow_df, metrics=["actual_duration"], title="multiple queries optimization",open_=False)
    

The total query duration is 3.5 seconds (from the “Total time” of the last node) we can see that it improved both table Scans on the Titles* relation.  We can continue to improve our query in the same manner if needed.

It’s important to understand that it’s not practical to take every pair of queries and do the same process. 